# 📚 Logistic Regression Complete Tutorial with E-commerce Dataset

## 🎯 Learning Objectives
1. Understand the difference between Linear and Logistic Regression
2. Master binary classification with single and multiple features
3. Learn multi-class classification techniques
4. Apply feature engineering and scaling
5. Evaluate models using appropriate metrics
6. Generate actionable business insights

## 📊 Dataset Overview
- **Source**: E-commerce Customers Dataset
- **Records**: 500 customers
- **Features**: Customer engagement metrics (session length, time on app/website, membership duration)
- **Target**: We'll create various classification targets from Yearly Amount Spent

---

## 🔧 Setup and Imports

In [4]:
# Install required libraries (run if needed)
# !pip install pandas numpy matplotlib seaborn plotly scikit-learn

# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Sklearn imports
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, learning_curve
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_curve, auc, 
    roc_auc_score, precision_recall_curve
)
from sklearn.calibration import calibration_curve
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.utils.class_weight import compute_class_weight

# Settings
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✅ All libraries imported successfully!')
print(f'Pandas version: {pd.__version__}')
print(f'NumPy version: {np.__version__}')

✅ All libraries imported successfully!
Pandas version: 2.2.2
NumPy version: 1.26.4


## 📁 Load and Explore Dataset

In [6]:
# Load the dataset
df = pd.read_csv('Ecommerce_Customers.csv')

# Basic information
print("📊 Dataset Overview")
print("="*50)
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"\nColumn Names and Types:")
print(df.dtypes)
print("\n" + "="*50)

📊 Dataset Overview
Shape: 500 rows, 8 columns

Column Names and Types:
Email                    object
Address                  object
Avatar                   object
Avg. Session Length     float64
Time on App             float64
Time on Website         float64
Length of Membership    float64
Yearly Amount Spent     float64
dtype: object



In [7]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

First 5 rows of the dataset:


,Email,Address,Avatar,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
0,mstephenson@fernandez.com,"835 Frank Tunnel\nWrightmouth, MI 82180-9605",Violet,34.50,12.66,39.58,4.08,587.95
1,hduke@hotmail.com,"4547 Archer Common\nDiazchester, CA 06566-8576",DarkGreen,31.93,11.11,37.27,2.66,392.20
2,pallen@yahoo.com,"24645 Valerie Unions Suite 582\nCobbborough, D...",Bisque,33.00,11.33,37.11,4.10,487.55
3,riverarebecca@gmail.com,"1414 David Throughway\nPort Jason, OH 22070-1220",SaddleBrown,34.31,13.72,36.72,3.12,581.85
4,mstephens@davidson-herman.com,"14023 Rodriguez Passage\nPort Jacobville, PR 3...",MediumAquaMarine,33.33,12.80,37.54,4.45,599.41


In [8]:
# Statistical summary
print("📈 Statistical Summary of Numerical Features")
df.describe()

📈 Statistical Summary of Numerical Features


,Avg. Session Length,Time on App,Time on Website,Length of Membership,Yearly Amount Spent
count,500.00,500.00,500.00,500.00,500.00
mean,33.05,12.05,37.06,3.53,499.31
std,0.99,0.99,1.01,1.00,79.31
min,29.53,8.51,33.91,0.27,256.67
25%,32.34,11.39,36.35,2.93,445.04
50%,33.08,11.98,37.07,3.53,498.89
75%,33.71,12.75,37.72,4.13,549.31
max,36.14,15.13,40.01,6.92,765.52


In [9]:
# Check for missing values
print("🔍 Missing Values Check:")
missing = df.isnull().sum()
if missing.sum() == 0:
    print("✅ No missing values found!")
else:
    print(missing[missing > 0])

🔍 Missing Values Check:
✅ No missing values found!


In [13]:
# Analyze target variable distribution
print("💰 Yearly Amount Spent Analysis")
print("="*50)
spending_stats = df['Yearly Amount Spent'].describe()
print(spending_stats)

# Calculate additional percentiles
print("\n📊 Spending Percentiles:")
for p in [25, 33, 50, 66, 75, 90]:
    val = df['Yearly Amount Spent'].quantile(p/100)
    print(f"{p:3d}th percentile: ${val:7.2f}")

💰 Yearly Amount Spent Analysis
count    500.00
mean     499.31
std       79.31
min      256.67
25%      445.04
50%      498.89
75%      549.31
max      765.52
Name: Yearly Amount Spent, dtype: float64

📊 Spending Percentiles:
 25th percentile: $ 445.04
 33th percentile: $ 467.70
 50th percentile: $ 498.89
 66th percentile: $ 532.33
 75th percentile: $ 549.31
 90th percentile: $ 593.23


In [14]:
# Visualize spending distribution
fig = px.histogram(df, x='Yearly Amount Spent', 
                   title='Distribution of Yearly Customer Spending',
                   nbins=30,
                   labels={'Yearly Amount Spent': 'Yearly Spending ($)'},
                   color_discrete_sequence=['#1f77b4'])

# Add mean and median lines
fig.add_vline(x=df['Yearly Amount Spent'].mean(), 
              line_dash="dash", line_color="red",
              annotation_text=f"Mean: ${df['Yearly Amount Spent'].mean():.0f}")
fig.add_vline(x=df['Yearly Amount Spent'].median(), 
              line_dash="dash", line_color="green",
              annotation_text=f"Median: ${df['Yearly Amount Spent'].median():.0f}")

fig.update_layout(height=400)
fig.show()

---
# 📊 Problem Statement 1: Binary Classification (Single Feature)

## Business Problem
**Identify High-Value Customers**: Predict whether a customer is a 'High Spender' (top 25% of customers) based on their **Length of Membership** alone.

### Why This Matters:
- Helps prioritize customer service resources
- Enables targeted marketing campaigns
- Identifies loyalty-value relationship

### Step 1: Create Binary Target Variable

In [15]:
# Create binary classification target
# High Spender = 1 if spending > 75th percentile, else 0

threshold_75 = df['Yearly Amount Spent'].quantile(0.75)
print(f"📍 High Spender Threshold (75th percentile): ${threshold_75:.2f}")

# Create binary target
df['High_Spender'] = (df['Yearly Amount Spent'] > threshold_75).astype(int)

# Check class distribution
class_distribution = df['High_Spender'].value_counts()
print("\n📊 Class Distribution:")
print(f"Regular Spenders (0): {class_distribution[0]} ({class_distribution[0]/len(df)*100:.1f}%)")
print(f"High Spenders (1):    {class_distribution[1]} ({class_distribution[1]/len(df)*100:.1f}%)")

# Visualize the distribution
fig = px.histogram(df, x='Yearly Amount Spent', 
                   color='High_Spender',
                   title='Distribution of Yearly Spending with High Spender Classification',
                   nbins=30,
                   labels={'High_Spender': 'Customer Type'},
                   color_discrete_map={0: '#3498db', 1: '#e74c3c'})

fig.add_vline(x=threshold_75, line_dash="dash", line_color="black",
              annotation_text=f"Threshold: ${threshold_75:.0f}")

fig.update_layout(height=400)
fig.show()

📍 High Spender Threshold (75th percentile): $549.31

📊 Class Distribution:
Regular Spenders (0): 375 (75.0%)
High Spenders (1):    125 (25.0%)


### Step 2: Prepare Features for Single Variable Model

In [ ]:
# Single feature: Length of Membership
X = df[['Length of Membership']]  # Keep as DataFrame
y = df['High_Spender']

print("📐 Feature and Target Shapes:")
print(f"X (features): {X.shape}")
print(f"y (target): {y.shape}")

# Visualize relationship
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=('Distribution by Customer Type', 
                                   'Scatter Plot with Jitter'))

# Box plot
for spender_type, name, color in [(0, 'Regular', '#3498db'), (1, 'High', '#e74c3c')]:
    subset = df[df['High_Spender'] == spender_type]['Length of Membership']
    fig.add_trace(go.Box(y=subset, name=name, marker_color=color), row=1, col=1)

# Scatter plot with jitter
jitter_strength = 0.05
df['High_Spender_Jittered'] = df['High_Spender'] + np.random.normal(0, jitter_strength, len(df))

fig.add_trace(go.Scatter(
    x=df['Length of Membership'],
    y=df['High_Spender_Jittered'],
    mode='markers',
    marker=dict(size=5, opacity=0.5, color=df['High_Spender'], 
                colorscale=[[0, '#3498db'], [1, '#e74c3c']]),
    showlegend=False
), row=1, col=2)

fig.update_xaxes(title_text="Customer Type", row=1, col=1)
fig.update_xaxes(title_text="Years of Membership", row=1, col=2)
fig.update_yaxes(title_text="Years of Membership", row=1, col=1)
fig.update_yaxes(title_text="High Spender (with jitter)", row=1, col=2)

fig.update_layout(height=400, title_text="Membership Length vs High Spender Status")
fig.show()

# Statistical summary
print("\n📊 Membership Length by Customer Type:")
for spender_type in [0, 1]:
    type_name = "Regular" if spender_type == 0 else "High   "
    subset = df[df['High_Spender'] == spender_type]['Length of Membership']
    print(f"{type_name} Spenders: Mean={subset.mean():.2f} years, Median={subset.median():.2f} years")

### Step 3: Split Data and Train Model

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, 
    stratify=y  # Important: Maintains class proportion in train/test
)

print("📊 Data Split Summary:")
print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nTraining set class distribution:")
print(y_train.value_counts(normalize=True).round(3))
print(f"\nTest set class distribution:")
print(y_test.value_counts(normalize=True).round(3))

In [ ]:
# Create and train logistic regression model
log_model = LogisticRegression(random_state=42)
log_model.fit(X_train, y_train)

# Get model parameters
print("🔧 Logistic Regression Model Parameters:")
print("="*50)
print(f"Coefficient (slope): {log_model.coef_[0][0]:.4f}")
print(f"Intercept: {log_model.intercept_[0]:.4f}")
print("\n📈 Interpretation:")

if log_model.coef_[0][0] > 0:
    print("✅ Positive coefficient: Longer membership → Higher probability of being high spender")
    print(f"   Each additional year increases log-odds by {log_model.coef_[0][0]:.4f}")
else:
    print("❌ Negative coefficient: Longer membership → Lower probability of being high spender")
    print(f"   Each additional year decreases log-odds by {abs(log_model.coef_[0][0]):.4f}")

### Step 4: Understanding Predictions and Probabilities

In [ ]:
# Make predictions
y_pred = log_model.predict(X_test)  # Binary predictions (0 or 1)
y_pred_proba = log_model.predict_proba(X_test)  # Probabilities for each class

# Create results DataFrame for inspection
results_df = pd.DataFrame({
    'Membership_Years': X_test['Length of Membership'].values,
    'Actual': y_test.values,
    'Predicted': y_pred,
    'Prob_Regular': y_pred_proba[:, 0],  # Probability of being regular spender
    'Prob_High': y_pred_proba[:, 1],      # Probability of being high spender
    'Correct': y_test.values == y_pred
})

# Sort by membership years for better visualization
results_df = results_df.sort_values('Membership_Years')

print("📋 Sample Predictions (sorted by membership years):")
print(results_df.head(10).to_string(index=False))

print("\n🎯 Edge Cases (probabilities close to 0.5):")
edge_cases = results_df[(results_df['Prob_High'] > 0.4) & (results_df['Prob_High'] < 0.6)]
if len(edge_cases) > 0:
    print(edge_cases.head().to_string(index=False))
else:
    print("No edge cases found with probability between 0.4 and 0.6")

### Step 5: Visualize Decision Boundary and Logistic Curve

In [ ]:
# Create visualization of logistic regression curve
# Generate points for smooth curve
x_range = np.linspace(X['Length of Membership'].min(), 
                     X['Length of Membership'].max(), 300)
x_range_df = pd.DataFrame(x_range, columns=['Length of Membership'])

# Get probabilities for the range
y_proba_range = log_model.predict_proba(x_range_df)[:, 1]

# Create interactive plot
fig = go.Figure()

# Add actual data points (test set)
for class_val, class_name, color in [(0, 'Regular Spender', '#3498db'), 
                                      (1, 'High Spender', '#e74c3c')]:
    mask = y_test == class_val
    fig.add_trace(go.Scatter(
        x=X_test[mask]['Length of Membership'],
        y=[class_val] * mask.sum(),
        mode='markers',
        name=class_name,
        marker=dict(size=8, color=color, opacity=0.5),
        yaxis='y2'
    ))

# Add logistic curve
fig.add_trace(go.Scatter(
    x=x_range,
    y=y_proba_range,
    mode='lines',
    name='Probability Curve',
    line=dict(color='green', width=3)
))

# Add decision boundary (0.5 probability)
fig.add_hline(y=0.5, line_dash="dash", line_color="black",
              annotation_text="Decision Boundary (p=0.5)")

# Find the decision point
decision_point = -log_model.intercept_[0] / log_model.coef_[0][0]
fig.add_vline(x=decision_point, line_dash="dot", line_color="purple",
              annotation_text=f"Decision at {decision_point:.2f} years")

# Update layout
fig.update_layout(
    title='Logistic Regression: Decision Boundary and Probability Curve',
    xaxis_title='Years of Membership',
    yaxis_title='Probability of Being High Spender',
    yaxis2=dict(title='Actual Class', overlaying='y', side='right', range=[-0.1, 1.1]),
    height=500,
    hovermode='x',
    showlegend=True
)
fig.show()

print(f"\n📍 Decision Boundary Analysis:")
print(f"Decision boundary at: {decision_point:.2f} years of membership")
print(f"Customers with >{decision_point:.2f} years are predicted as High Spenders")

### Step 6: Evaluate Model Performance

In [ ]:
# Calculate all metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("📊 Model Performance Metrics")
print("="*60)
print(f"Accuracy:  {accuracy:.3f} - Overall correct predictions")
print(f"Precision: {precision:.3f} - When predicting High Spender, how often correct")
print(f"Recall:    {recall:.3f} - Of all High Spenders, how many we caught")
print(f"F1 Score:  {f1:.3f} - Harmonic mean of Precision and Recall")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Create interactive confusion matrix
fig = px.imshow(cm, 
                labels=dict(x="Predicted", y="Actual"),
                x=['Regular Spender', 'High Spender'],
                y=['Regular Spender', 'High Spender'],
                color_continuous_scale='Blues',
                text_auto=True,
                title='Confusion Matrix')
fig.update_layout(height=400)
fig.show()

# Detailed interpretation
print("\n🔍 Confusion Matrix Interpretation:")
print(f"True Negatives (TN):  {cm[0,0]:3d} - Correctly identified Regular Spenders")
print(f"False Positives (FP): {cm[0,1]:3d} - Regular incorrectly labeled as High")
print(f"False Negatives (FN): {cm[1,0]:3d} - High incorrectly labeled as Regular")
print(f"True Positives (TP):  {cm[1,1]:3d} - Correctly identified High Spenders")

# Business impact analysis
print("\n💰 Business Impact Analysis (Hypothetical):")
cost_false_negative = 100  # Missing a high spender costs $100 in lost opportunity
cost_false_positive = 20   # Misclassifying regular as high costs $20 in wasted resources

total_cost = cm[1,0] * cost_false_negative + cm[0,1] * cost_false_positive
print(f"Cost of False Negatives: ${cm[1,0] * cost_false_negative:,}")
print(f"Cost of False Positives: ${cm[0,1] * cost_false_positive:,}")
print(f"Total Misclassification Cost: ${total_cost:,}")

### Step 7: ROC Curve and AUC Analysis

In [ ]:
# ROC Curve shows trade-off between True Positive Rate and False Positive Rate
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])
roc_auc = auc(fpr, tpr)

# Create interactive ROC curve
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=fpr, y=tpr,
    mode='lines',
    name=f'ROC Curve (AUC = {roc_auc:.3f})',
    line=dict(color='blue', width=2),
    fill='tonexty'
))

# Add diagonal line (random classifier)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random Classifier',
    line=dict(color='red', width=1, dash='dash')
))

fig.update_layout(
    title='ROC Curve: Model Performance',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    width=600, height=600,
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1])
)
fig.show()

print(f"📈 AUC Score: {roc_auc:.3f}")
print("\n📚 AUC Interpretation Guide:")
print("0.5      = Random guessing")
print("0.7-0.8  = Acceptable")
print("0.8-0.9  = Excellent")
">0.9      = Outstanding (check for overfitting!)")

# Classify our model performance
if roc_auc < 0.7:
    print(f"\n⚠️ Model performance is poor (AUC = {roc_auc:.3f})")
elif roc_auc < 0.8:
    print(f"\n✅ Model performance is acceptable (AUC = {roc_auc:.3f})")
elif roc_auc < 0.9:
    print(f"\n🎯 Model performance is excellent (AUC = {roc_auc:.3f})")
else:
    print(f"\n🏆 Model performance is outstanding (AUC = {roc_auc:.3f})")

---
# 📊 Problem Statement 2: Multi-Feature Binary Classification

## Business Problem
**Customer Retention Risk**: Predict if a customer is at risk of becoming a low-value customer (bottom 33%) using multiple engagement metrics.

### Why This Matters:
- Early identification of at-risk customers
- Targeted retention campaigns
- Resource optimization

### Step 1: Create Target and Select Features

In [ ]:
# Create new target: At-Risk Customer (bottom 33%)
threshold_33 = df['Yearly Amount Spent'].quantile(0.33)
df['At_Risk'] = (df['Yearly Amount Spent'] < threshold_33).astype(int)

print(f"📍 At-Risk threshold (33rd percentile): ${threshold_33:.2f}")
print(f"At-Risk customers: {df['At_Risk'].sum()} ({df['At_Risk'].mean()*100:.1f}%)")
print(f"Safe customers: {(1-df['At_Risk']).sum()} ({(1-df['At_Risk']).mean()*100:.1f}%)")

# Select multiple features
feature_cols = ['Avg. Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
X = df[feature_cols]
y = df['At_Risk']

# Check correlation with target
correlations = pd.DataFrame({
    'Feature': feature_cols,
    'Correlation_with_AtRisk': [df[col].corr(df['At_Risk']) for col in feature_cols]
}).sort_values('Correlation_with_AtRisk')

print("\n📊 Feature Correlations with At-Risk Status:")
print(correlations.to_string(index=False))

# Visualize correlations
fig = px.bar(correlations, x='Correlation_with_AtRisk', y='Feature',
             orientation='h',
             title='Feature Correlations with At-Risk Status',
             color='Correlation_with_AtRisk',
             color_continuous_scale='RdBu_r')
fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.update_layout(height=300)
fig.show()

In [ ]:
# Visualize feature distributions by class
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=feature_cols)

for idx, col in enumerate(feature_cols):
    row = idx // 2 + 1
    col_pos = idx % 2 + 1
    
    # Add box plots for each class
    for at_risk, name, color in [(0, 'Safe', '#2ecc71'), (1, 'At-Risk', '#e74c3c')]:
        subset = df[df['At_Risk'] == at_risk][col]
        fig.add_trace(go.Box(y=subset, name=name, marker_color=color,
                            showlegend=(idx==0)),
                     row=row, col=col_pos)

fig.update_layout(height=600, title_text="Feature Distributions by At-Risk Status")
fig.show()

### Step 2: Scale Features (Important for Multiple Features!)

In [ ]:
# Split data first
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
# Why? Features have different scales (minutes, years, etc.)
# Logistic regression uses gradient descent which is sensitive to scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

print("🔄 Feature Scaling Applied")
print("\nFeatures before scaling (first row):")
print(X_train.iloc[0].to_frame().T)
print("\nFeatures after scaling (first row):")
print(X_train_scaled.iloc[0].to_frame().T)
print("\n📊 Scaling Statistics:")
print("Mean after scaling:", X_train_scaled.mean().round(10).values)
print("Std after scaling:", X_train_scaled.std().values)

### Step 3: Train Multi-Feature Model

In [ ]:
# Train logistic regression with multiple features
log_model_multi = LogisticRegression(random_state=42, max_iter=1000)
log_model_multi.fit(X_train_scaled, y_train)

# Get feature importance (coefficients)
feature_importance = pd.DataFrame({
    'Feature': feature_cols,
    'Coefficient': log_model_multi.coef_[0],
    'Abs_Coefficient': np.abs(log_model_multi.coef_[0])
}).sort_values('Abs_Coefficient', ascending=False)

print("🎯 Feature Importance (Coefficients):")
print(feature_importance.to_string(index=False))

# Visualize feature importance
fig = px.bar(feature_importance, 
             x='Coefficient', y='Feature',
             orientation='h',
             title='Feature Importance: What Predicts At-Risk Customers?',
             color='Coefficient',
             color_continuous_scale='RdBu_r',
             labels={'Coefficient': 'Impact on At-Risk Probability'})
fig.add_vline(x=0, line_dash="dash", line_color="black")
fig.update_layout(height=400)
fig.show()

# Interpretation
print("\n📈 Coefficient Interpretation:")
for _, row in feature_importance.iterrows():
    direction = "increases" if row['Coefficient'] > 0 else "decreases"
    print(f"• {row['Feature']:25s}: Higher values {direction} at-risk probability")

### Step 4: Compare Single vs Multiple Features

In [ ]:
# Train single feature model for comparison
X_train_single = X_train[['Length of Membership']]
X_test_single = X_test[['Length of Membership']]

log_model_single = LogisticRegression(random_state=42)
log_model_single.fit(X_train_single, y_train)

# Get predictions for both models
y_pred_single = log_model_single.predict(X_test_single)
y_pred_multi = log_model_multi.predict(X_test_scaled)

y_proba_single = log_model_single.predict_proba(X_test_single)[:, 1]
y_proba_multi = log_model_multi.predict_proba(X_test_scaled)[:, 1]

# Calculate metrics for both
print("="*60)
print("SINGLE FEATURE MODEL (Length of Membership only)")
print("="*60)
print(classification_report(y_test, y_pred_single, 
                           target_names=['Not At-Risk', 'At-Risk']))

print("\n" + "="*60)
print("MULTI-FEATURE MODEL (All engagement metrics)")
print("="*60)
print(classification_report(y_test, y_pred_multi,
                           target_names=['Not At-Risk', 'At-Risk']))

In [ ]:
# ROC Curves comparison
fig = go.Figure()

# Single feature ROC
fpr_single, tpr_single, _ = roc_curve(y_test, y_proba_single)
auc_single = roc_auc_score(y_test, y_proba_single)
fig.add_trace(go.Scatter(x=fpr_single, y=tpr_single,
                        name=f'Single Feature (AUC={auc_single:.3f})',
                        mode='lines',
                        line=dict(color='orange', width=2)))

# Multi feature ROC
fpr_multi, tpr_multi, _ = roc_curve(y_test, y_proba_multi)
auc_multi = roc_auc_score(y_test, y_proba_multi)
fig.add_trace(go.Scatter(x=fpr_multi, y=tpr_multi,
                        name=f'Multi Feature (AUC={auc_multi:.3f})',
                        mode='lines',
                        line=dict(color='blue', width=2)))

# Random classifier
fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1],
                        mode='lines', line=dict(dash='dash', color='red'),
                        name='Random Classifier'))

fig.update_layout(title='ROC Curves: Single vs Multi Feature Models',
                 xaxis_title='False Positive Rate',
                 yaxis_title='True Positive Rate',
                 width=600, height=600)
fig.show()

improvement = (auc_multi - auc_single) * 100
print(f"\n📈 AUC Improvement with Multiple Features: {improvement:.1f}%")
print(f"Single Feature AUC: {auc_single:.3f}")
print(f"Multi Feature AUC:  {auc_multi:.3f}")

### Step 5: Find Optimal Threshold

In [ ]:
# Calculate precision-recall for different thresholds
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_multi)

# Calculate F1 scores for each threshold
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-10)

# Find optimal threshold
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx] if optimal_idx < len(thresholds) else 0.5

print(f"🎯 Threshold Optimization:")
print(f"Default threshold: 0.5")
print(f"Optimal threshold: {optimal_threshold:.3f}")
print(f"F1 at optimal threshold: {f1_scores[optimal_idx]:.3f}")

# Compare predictions with different thresholds
y_pred_default = (y_proba_multi >= 0.5).astype(int)
y_pred_optimal = (y_proba_multi >= optimal_threshold).astype(int)

print("\n" + "="*50)
print("Performance with DEFAULT threshold (0.5):")
print("="*50)
print(f"Accuracy:  {accuracy_score(y_test, y_pred_default):.3f}")
print(f"Precision: {precision_score(y_test, y_pred_default):.3f}")
print(f"Recall:    {recall_score(y_test, y_pred_default):.3f}")

print("\n" + "="*50)
print(f"Performance with OPTIMAL threshold ({optimal_threshold:.3f}):")
print("="*50)
print(f"Accuracy:  {accuracy_score(y_test, y_pred_optimal):.3f}")
print(f"Precision: {precision_score(y_test, y_pred_optimal):.3f}")
print(f"Recall:    {recall_score(y_test, y_pred_optimal):.3f}")

In [ ]:
# Visualize threshold impact
fig = go.Figure()
fig.add_trace(go.Scatter(x=thresholds, y=precision[:-1],
                        mode='lines', name='Precision',
                        line=dict(color='blue')))
fig.add_trace(go.Scatter(x=thresholds, y=recall[:-1],
                        mode='lines', name='Recall',
                        line=dict(color='green')))
fig.add_trace(go.Scatter(x=thresholds, y=f1_scores[:-1],
                        mode='lines', name='F1 Score',
                        line=dict(color='red', width=3)))
fig.add_vline(x=optimal_threshold, line_dash="dash", line_color="black",
             annotation_text=f"Optimal: {optimal_threshold:.3f}")
fig.update_layout(title='Metrics vs Decision Threshold',
                 xaxis_title='Threshold',
                 yaxis_title='Score',
                 hovermode='x',
                 height=400)
fig.show()

---
# 📊 Problem Statement 3: Multi-Class Classification with Feature Engineering

## Business Problem
**Customer Segmentation**: Classify customers into 3 segments (Bronze, Silver, Gold) for targeted marketing.

### Why This Matters:
- Personalized marketing campaigns
- Tiered service levels
- Resource allocation optimization

### Step 1: Create Multi-Class Target

In [ ]:
# Create 3 customer segments based on spending
spending_tertiles = df['Yearly Amount Spent'].quantile([0.33, 0.66])

def classify_customer(spending):
    if spending <= spending_tertiles[0.33]:
        return 0  # Bronze
    elif spending <= spending_tertiles[0.66]:
        return 1  # Silver
    else:
        return 2  # Gold

df['Customer_Segment'] = df['Yearly Amount Spent'].apply(classify_customer)

# Map to readable labels
segment_map = {0: 'Bronze', 1: 'Silver', 2: 'Gold'}
df['Segment_Name'] = df['Customer_Segment'].map(segment_map)

# Check distribution
print("🏅 Customer Segmentation:")
print(df['Segment_Name'].value_counts().sort_index())
print("\n💰 Spending ranges by segment:")
for segment in [0, 1, 2]:
    segment_data = df[df['Customer_Segment'] == segment]['Yearly Amount Spent']
    print(f"{segment_map[segment]:6s}: ${segment_data.min():7.2f} - ${segment_data.max():7.2f} (Mean: ${segment_data.mean():7.2f})")

# Visualize segments
fig = px.box(df, x='Segment_Name', y='Yearly Amount Spent',
             title='Customer Segments Distribution',
             color='Segment_Name',
             color_discrete_map={'Bronze': '#CD7F32', 'Silver': '#C0C0C0', 'Gold': '#FFD700'})
fig.update_layout(height=400)
fig.show()

### Step 2: Feature Engineering

In [ ]:
# Create new features to improve classification
df_eng = df.copy()

# New Feature 1: Total Digital Engagement
df_eng['Total_Digital_Time'] = df_eng['Time on App'] + df_eng['Time on Website']

# New Feature 2: Engagement Intensity
df_eng['Engagement_Intensity'] = (
    df_eng['Avg. Session Length'] * 
    (df_eng['Time on App'] + df_eng['Time on Website']) / 100
)

# New Feature 3: App Preference Ratio
df_eng['App_Preference'] = (
    df_eng['Time on App'] / 
    (df_eng['Time on App'] + df_eng['Time on Website'] + 1e-10)
)

# New Feature 4: Platform Preference (categorical → numerical)
def platform_preference(row):
    app_ratio = row['App_Preference']
    if app_ratio < 0.3:
        return 0  # Web preferred
    elif app_ratio < 0.7:
        return 1  # Balanced
    else:
        return 2  # App preferred

df_eng['Platform_Preference'] = df_eng.apply(platform_preference, axis=1)

# New Feature 5: Customer Lifetime Value proxy
df_eng['CLV_Proxy'] = df_eng['Length of Membership'] * df_eng['Avg. Session Length']

# New Feature 6: Session Efficiency
df_eng['Session_Efficiency'] = (
    (df_eng['Time on App'] + df_eng['Time on Website']) / 
    (df_eng['Avg. Session Length'] + 1e-10)
)

# New Feature 7: Digital Adoption Score
df_eng['Digital_Adoption'] = (
    (df_eng['Time on App'] - df_eng['Time on App'].mean()) / df_eng['Time on App'].std() +
    (df_eng['Time on Website'] - df_eng['Time on Website'].mean()) / df_eng['Time on Website'].std()
)

# List all features
original_features = ['Avg. Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
engineered_features = ['Total_Digital_Time', 'Engagement_Intensity', 'App_Preference',
                       'Platform_Preference', 'CLV_Proxy', 'Session_Efficiency', 'Digital_Adoption']
all_features = original_features + engineered_features

print("🔧 Feature Engineering Complete!")
print(f"Original features: {len(original_features)}")
print(f"Engineered features: {len(engineered_features)}")
print(f"Total features: {len(all_features)}")

# Check new features
print("\n📊 Engineered Features Sample:")
print(df_eng[engineered_features].head())

In [ ]:
# Visualize engineered features vs target
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=['Total Digital Time', 'Engagement Intensity',
                                   'CLV Proxy', 'Digital Adoption'])

features_to_plot = ['Total_Digital_Time', 'Engagement_Intensity', 'CLV_Proxy', 'Digital_Adoption']

for idx, feat in enumerate(features_to_plot):
    row = idx // 2 + 1
    col = idx % 2 + 1
    
    for segment, name, color in [(0, 'Bronze', '#CD7F32'), 
                                 (1, 'Silver', '#C0C0C0'), 
                                 (2, 'Gold', '#FFD700')]:
        subset = df_eng[df_eng['Customer_Segment'] == segment][feat]
        fig.add_trace(go.Box(y=subset, name=name, marker_color=color,
                            showlegend=(idx==0)),
                     row=row, col=col)

fig.update_layout(height=600, title_text="Engineered Features by Customer Segment")
fig.show()

### Step 3: Prepare Data and Train Multi-Class Model

In [ ]:
# Prepare features and target
X_all = df_eng[all_features]
y = df_eng['Customer_Segment']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_all, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Keep column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=all_features, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=all_features, index=X_test.index)

print("📊 Data prepared for multi-class classification")
print(f"Training samples per class:")
print(y_train.value_counts().sort_index())
print(f"\nTest samples per class:")
print(y_test.value_counts().sort_index())

In [ ]:
# Train both One-vs-Rest and Multinomial models

# Model 1: One-vs-Rest
log_multi_ovr = LogisticRegression(
    multi_class='ovr',  # One vs Rest strategy
    random_state=42,
    max_iter=1000
)
log_multi_ovr.fit(X_train_scaled, y_train)

# Model 2: Multinomial (Softmax)
log_multi_mn = LogisticRegression(
    multi_class='multinomial',  # Multinomial (softmax)
    random_state=42,
    max_iter=1000
)
log_multi_mn.fit(X_train_scaled, y_train)

# Get predictions
y_pred_ovr = log_multi_ovr.predict(X_test_scaled)
y_pred_mn = log_multi_mn.predict(X_test_scaled)

# Get probabilities
y_proba_ovr = log_multi_ovr.predict_proba(X_test_scaled)
y_proba_mn = log_multi_mn.predict_proba(X_test_scaled)

print("✅ Models trained with both strategies!")

### Step 4: Evaluate Multi-Class Performance

In [ ]:
# Compare both approaches
print("="*60)
print("ONE-VS-REST APPROACH")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_ovr):.3f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred_ovr, 
                           target_names=['Bronze', 'Silver', 'Gold']))

print("\n" + "="*60)
print("MULTINOMIAL APPROACH")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred_mn):.3f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred_mn,
                           target_names=['Bronze', 'Silver', 'Gold']))

In [ ]:
# Confusion Matrix for best model
cm = confusion_matrix(y_test, y_pred_mn)

# Visualize confusion matrix
fig = px.imshow(cm,
                labels=dict(x="Predicted Segment", y="Actual Segment"),
                x=['Bronze', 'Silver', 'Gold'],
                y=['Bronze', 'Silver', 'Gold'],
                color_continuous_scale='Viridis',
                text_auto=True,
                title='Multi-Class Confusion Matrix (Multinomial Model)')
fig.update_layout(height=400)
fig.show()

# Analyze misclassifications
print("\n🔍 Misclassification Analysis:")
for i in range(3):
    for j in range(3):
        if i != j and cm[i, j] > 0:
            print(f"{segment_map[i]:6s} misclassified as {segment_map[j]:6s}: {cm[i, j]:2d} cases")

### Step 5: Feature Importance Analysis

In [ ]:
# Get coefficients for each class (One-vs-Rest gives us this)
feature_importance_by_class = pd.DataFrame(
    log_multi_ovr.coef_.T,
    columns=['Bronze', 'Silver', 'Gold'],
    index=all_features
)

# Create subplots for each segment
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=['Bronze Segment', 'Silver Segment', 'Gold Segment'])

for idx, segment in enumerate(['Bronze', 'Silver', 'Gold']):
    importance = feature_importance_by_class[segment].sort_values()
    colors = ['red' if x < 0 else 'green' for x in importance.values]
    
    fig.add_trace(go.Bar(
        x=importance.values,
        y=importance.index,
        orientation='h',
        marker_color=colors,
        name=segment,
        showlegend=False
    ), row=1, col=idx+1)

fig.update_layout(height=500, title_text="Feature Importance by Customer Segment")
fig.show()

# Top features per segment
print("🎯 Top 3 Features per Customer Segment:")
for segment in ['Bronze', 'Silver', 'Gold']:
    top_features = feature_importance_by_class[segment].abs().nlargest(3)
    print(f"\n{segment}:")
    for feat, coef_abs in top_features.items():
        coef = feature_importance_by_class[segment][feat]
        direction = "positive" if coef > 0 else "negative"
        print(f"  • {feat:25s}: {direction} impact (coef={coef:6.3f})")

### Step 6: Cross-Validation Analysis

In [ ]:
# Use StratifiedKFold to maintain class proportions
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross-validate both original and engineered features
# Original features only
X_original = df_eng[original_features]
X_original_scaled = scaler.fit_transform(X_original)

cv_scores_original = cross_val_score(
    LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42),
    X_original_scaled, y,
    cv=cv_strategy,
    scoring='accuracy'
)

# All features (original + engineered)
X_all_scaled = scaler.fit_transform(X_all)
cv_scores_all = cross_val_score(
    LogisticRegression(multi_class='multinomial', max_iter=1000, random_state=42),
    X_all_scaled, y,
    cv=cv_strategy,
    scoring='accuracy'
)

print("📊 Cross-Validation Results (5-fold):")
print("="*50)
print("Original Features Only:")
print(f"  Scores: {cv_scores_original.round(3)}")
print(f"  Mean: {cv_scores_original.mean():.3f} (+/- {cv_scores_original.std()*2:.3f})")

print("\nOriginal + Engineered Features:")
print(f"  Scores: {cv_scores_all.round(3)}")
print(f"  Mean: {cv_scores_all.mean():.3f} (+/- {cv_scores_all.std()*2:.3f})")

improvement = (cv_scores_all.mean() - cv_scores_original.mean()) * 100
print(f"\n📈 Improvement from Feature Engineering: {improvement:.1f}%")

---
# 🎯 Advanced Techniques

### Regularized Logistic Regression

In [ ]:
# LogisticRegressionCV automatically finds best regularization parameter
log_reg_cv = LogisticRegressionCV(
    Cs=10,  # Test 10 different C values
    cv=5,   # 5-fold cross-validation
    penalty='l2',  # L2 regularization (Ridge)
    multi_class='multinomial',
    max_iter=1000,
    random_state=42
)

log_reg_cv.fit(X_train_scaled, y_train)

print("🔧 Regularization Analysis:")
print(f"Best C parameter: {log_reg_cv.C_[0]:.4f}")
print(f"Regularization strength: {1/log_reg_cv.C_[0]:.4f}")

# Compare with unregularized model
y_pred_regularized = log_reg_cv.predict(X_test_scaled)
acc_regularized = accuracy_score(y_test, y_pred_regularized)
acc_unregularized = accuracy_score(y_test, y_pred_mn)

print(f"\n📊 Performance Comparison:")
print(f"Accuracy without regularization: {acc_unregularized:.3f}")
print(f"Accuracy with regularization:    {acc_regularized:.3f}")

# Check coefficient shrinkage
coef_regularized = np.abs(log_reg_cv.coef_).mean(axis=0)
coef_unregularized = np.abs(log_multi_mn.coef_).mean(axis=0)

shrinkage = ((coef_unregularized - coef_regularized) / coef_unregularized * 100)

print("\n📉 Average Coefficient Shrinkage from Regularization:")
for i, feat in enumerate(all_features):
    print(f"{feat:25s}: {shrinkage[i]:6.1f}% shrinkage")

---
# 📝 Business Insights & Recommendations

In [ ]:
# Generate actionable insights
print("="*60)
print("BUSINESS INSIGHTS FROM LOGISTIC REGRESSION ANALYSIS")
print("="*60)

# High Spender Insights
if log_model.coef_[0][0] > 0:
    print("\n1. CUSTOMER LOYALTY DRIVES HIGH SPENDING ✅")
    print(f"   • Each year of membership increases high-spender probability")
    print(f"   • Decision boundary: {-log_model.intercept_[0] / log_model.coef_[0][0]:.2f} years")
    print("   → Action: Invest in retention programs and loyalty rewards")

# At-Risk Customer Insights
print("\n2. AT-RISK CUSTOMER EARLY WARNING SIGNALS ⚠️")
at_risk_top = feature_importance.nlargest(2, 'Abs_Coefficient')
print(f"   • Top risk indicators: {', '.join(at_risk_top['Feature'].tolist())}")
print("   → Action: Monitor these metrics for early intervention")

# Segmentation Insights
print("\n3. CUSTOMER SEGMENT CHARACTERISTICS 🏅")
for segment in ['Bronze', 'Silver', 'Gold']:
    top_feature = feature_importance_by_class[segment].abs().idxmax()
    print(f"   • {segment:6s}: Most influenced by '{top_feature}'")
print("   → Action: Customize marketing messages per segment")

# Model Performance
print(f"\n4. PREDICTION RELIABILITY 📊")
print(f"   • Binary classification AUC: {max(auc_single, auc_multi):.3f}")
print(f"   • Multi-class accuracy: {max(acc_regularized, acc_unregularized):.1%}")
print(f"   • Feature engineering improved accuracy by {improvement:.1f}%")
print("   → Action: Models are production-ready for customer targeting")

# Threshold Optimization
if optimal_threshold != 0.5:
    print(f"\n5. CLASSIFICATION THRESHOLD OPTIMIZATION 🎯")
    print(f"   • Optimal threshold: {optimal_threshold:.3f} (vs default 0.5)")
    print("   → Action: Adjust system thresholds for better precision/recall balance")

print("\n" + "="*60)

---
# 🎓 Key Takeaways

## Logistic Regression Mastery Checklist:

### ✅ Concepts Learned:
1. **Binary Classification**: Predicting two classes (0/1)
2. **Multi-Class Classification**: Predicting 3+ classes
3. **Probability Output**: Understanding probabilities vs predictions
4. **Decision Boundaries**: How threshold affects predictions
5. **Feature Scaling**: Critical for gradient-based optimization
6. **Feature Engineering**: Creating meaningful predictors
7. **Model Evaluation**: ROC curves, AUC, confusion matrices
8. **Regularization**: Preventing overfitting

### 📊 Model Performance Summary:
- Single Feature Binary: Basic baseline
- Multi-Feature Binary: Improved with more information
- Multi-Class: Successful customer segmentation
- Feature Engineering: Significant performance boost

### 🚀 Next Steps:
1. **Decision Trees**: Non-linear classification
2. **Random Forests**: Ensemble methods
3. **SVM**: Support Vector Machines
4. **Neural Networks**: Deep learning
5. **Clustering**: Unsupervised learning

---

## 📚 Practice Exercises:

1. **Change the threshold**: Try different percentiles for high spenders (top 10%, 20%, etc.)
2. **Create new features**: Engineer domain-specific features
3. **Try different regularization**: Compare L1 (Lasso) vs L2 (Ridge)
4. **Implement cost-sensitive learning**: Add class weights for imbalanced data
5. **Build a prediction pipeline**: Create end-to-end prediction system

---

**Happy Learning! 🎉**